# Árboles de Regresión

Vamos a hacer un árbol para cada target

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import time
from sklearn.metrics import mean_absolute_error

In [2]:
tiempo = pd.read_parquet('dataframe_final_1970-2023_alicante_alicante.parquet')

In [3]:
tiempo

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,horaPresMax_3,horaPresMax_mañana,presMin_1,presMin_2,presMin_3,presMin_mañana,horaPresMin_1,horaPresMin_2,horaPresMin_3,horaPresMin_mañana
0,1970-01-01,8025,ALACANT/ALICANTE,ALICANTE,81,9.3,0.0,5.0,01:15:00,13.6,...,None,10:00:00,None,None,None,998.1,None,None,None,19:00:00
1,1970-01-02,8025,ALACANT/ALICANTE,ALICANTE,81,7.0,7.8,3.6,05:45:00,10.4,...,None,00:00:00,993.0,None,None,995.4,00:00:00,None,None,None
2,1970-01-03,8025,ALACANT/ALICANTE,ALICANTE,81,11.1,0.0,7.4,00:45:00,14.8,...,None,00:00:00,998.1,993.0,None,989.3,19:00:00,00:00:00,None,15:00:00
3,1970-01-04,8025,ALACANT/ALICANTE,ALICANTE,81,13.8,0.2,9.0,05:20:00,18.6,...,None,10:00:00,995.4,998.1,993.0,988.8,None,19:00:00,00:00:00,None
4,1970-01-05,8025,ALACANT/ALICANTE,ALICANTE,81,14.6,0.5,9.8,05:30:00,19.4,...,10:00:00,None,989.3,995.4,998.1,986.9,15:00:00,None,19:00:00,07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19353,2022-12-27,8025,ALACANT/ALICANTE,ALICANTE,81,13.3,0.0,6.6,07:00:00,20.0,...,None,00:00:00,1013.3,1010.4,1010.4,1013.8,00:00:00,05:00:00,14:00:00,None
19354,2022-12-28,8025,ALACANT/ALICANTE,ALICANTE,81,12.7,0.0,6.1,04:30:00,19.3,...,23:00:00,10:00:00,1018.5,1013.3,1010.4,1012.7,01:00:00,00:00:00,05:00:00,14:00:00
19355,2022-12-29,8025,ALACANT/ALICANTE,ALICANTE,81,14.2,0.0,6.6,02:10:00,21.9,...,23:00:00,23:00:00,1013.8,1018.5,1013.3,1013.9,None,01:00:00,00:00:00,None
19356,2022-12-30,8025,ALACANT/ALICANTE,ALICANTE,81,17.2,0.0,10.5,03:50:00,24.0,...,11:00:00,None,1012.7,1013.8,1018.5,1015.4,14:00:00,None,01:00:00,15:00:00


In [3]:
# 70% del tiempo para el train, 30% validacion (hasta 31 diciembre 2022)
# el conjunto de test cogeremos 2023

fecha_test = '2007-12-31'

In [4]:
# las variables del día siguiente las eliminamos y también las correspondientes a la localización ya que estamos solo en Alicante

model_columns = list(set(tiempo.columns) - set(['tmed_mañana', 'prec_mañana', 'tmin_mañana', 'horatmin_mañana', 'tmax_mañana',
                                                                                    'horatmax_mañana', 'dir_mañana', 'velmedia_mañana', 'racha_mañana', 'horaracha_mañana', 
                                                                                    'sol_mañana', 'presMax_mañana', 'horaPresMax_mañana', 'presMin_mañana', 'horaPresMin_mañana',
                                                                                    'indicativo', 'nombre', 'provincia', 'altitud', 'fecha']))

In [5]:
tiempo.horatmin = tiempo.horatmin.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax = tiempo.horatmax.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha = tiempo.horaracha.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaPresMax = tiempo.horaPresMax.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin = tiempo.horaPresMin.apply(lambda x: x.hour if x is not None else np.nan)

tiempo.horatmin_1 = tiempo.horatmin_1.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmin_2 = tiempo.horatmin_2.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmin_3 = tiempo.horatmin_3.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmin_mañana = tiempo.horatmin_mañana.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)

tiempo.horatmax_1 = tiempo.horatmax_1.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax_2 = tiempo.horatmax_2.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax_3 = tiempo.horatmax_3.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax_mañana = tiempo.horatmax_mañana.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)

tiempo.horaracha_1 = tiempo.horaracha_1.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha_2 = tiempo.horaracha_2.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha_3 = tiempo.horaracha_3.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha_mañana = tiempo.horaracha_mañana.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)

tiempo.horaPresMax_1 = tiempo.horaPresMax_1.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMax_2 = tiempo.horaPresMax_2.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMax_3 = tiempo.horaPresMax_3.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMax_mañana = tiempo.horaPresMax_mañana.apply(lambda x: x.hour if x is not None else np.nan)

tiempo.horaPresMin_1 = tiempo.horaPresMin_1.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin_2 = tiempo.horaPresMin_2.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin_3 = tiempo.horaPresMin_3.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin_mañana = tiempo.horaPresMin_mañana.apply(lambda x: x.hour if x is not None else np.nan)

In [6]:
tiempo['año'] = tiempo['fecha'].dt.year
tiempo['mes'] = tiempo['fecha'].dt.month
tiempo['dia'] = tiempo['fecha'].dt.day

In [7]:
train = tiempo[tiempo.fecha <= fecha_test].fillna(-1)
test = tiempo[tiempo.fecha > fecha_test].fillna(-1)

In [9]:
train

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,presMin_2,presMin_3,presMin_mañana,horaPresMin_1,horaPresMin_2,horaPresMin_3,horaPresMin_mañana,año,mes,dia
0,1970-01-01,8025,ALACANT/ALICANTE,ALICANTE,81,9.3,0.0,5.0,1.0,13.6,...,-1,-1,998.1,-1.0,-1.0,-1.0,19.0,1970,1,1
1,1970-01-02,8025,ALACANT/ALICANTE,ALICANTE,81,7.0,7.8,3.6,6.0,10.4,...,-1,-1,995.4,0.0,-1.0,-1.0,-1.0,1970,1,2
2,1970-01-03,8025,ALACANT/ALICANTE,ALICANTE,81,11.1,0.0,7.4,1.0,14.8,...,993.0,-1,989.3,19.0,0.0,-1.0,15.0,1970,1,3
3,1970-01-04,8025,ALACANT/ALICANTE,ALICANTE,81,13.8,0.2,9.0,5.0,18.6,...,998.1,993.0,988.8,-1.0,19.0,0.0,-1.0,1970,1,4
4,1970-01-05,8025,ALACANT/ALICANTE,ALICANTE,81,14.6,0.5,9.8,6.0,19.4,...,995.4,998.1,986.9,15.0,-1.0,19.0,7.0,1970,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13874,2007-12-27,8025,ALACANT/ALICANTE,ALICANTE,81,11.3,0.0,6.8,0.0,15.8,...,1013.9,1013.1,1019.5,5.0,17.0,2.0,16.0,2007,12,27
13875,2007-12-28,8025,ALACANT/ALICANTE,ALICANTE,81,9.9,0.0,4.0,6.0,15.8,...,1013.5,1013.9,1018.8,0.0,5.0,17.0,-1.0,2007,12,28
13876,2007-12-29,8025,ALACANT/ALICANTE,ALICANTE,81,9.4,0.0,2.5,8.0,16.2,...,1018.6,1013.5,1014.9,16.0,0.0,5.0,16.0,2007,12,29
13877,2007-12-30,8025,ALACANT/ALICANTE,ALICANTE,81,11.1,0.0,3.4,6.0,18.8,...,1019.5,1018.6,1014.6,-1.0,16.0,0.0,5.0,2007,12,30


In [10]:
train[model_columns]

,tmed_3,tmax_2,presMax_1,tmin,dir_2,sol_3,racha_3,tmax,sol_1,horatmax_2,...,tmin_1,horaPresMin_2,horatmin,presMax_3,horaPresMax_2,presMin_2,tmin_3,horaPresMin_1,tmax_3,presMin_3
0,-1,-1,-1,5.0,-1.0,-1,-1,13.6,-1,-1.0,...,-1,-1.0,1.0,-1,-1.0,-1,-1,-1.0,-1,-1
1,-1,-1,1000.6,3.6,-1.0,-1,-1,10.4,6.4,-1.0,...,5.0,-1.0,6.0,-1,-1.0,-1,-1,0.0,-1,-1
2,-1,13.6,1001.0,7.4,32.0,-1,-1,14.8,0.0,14.0,...,3.6,0.0,1.0,-1,-1.0,993.0,-1,19.0,-1,-1
3,9.3,10.4,998.2,9.0,5.0,6.4,11.1,18.6,3.5,12.0,...,7.4,19.0,5.0,1000.6,10.0,998.1,5.0,-1.0,13.6,993.0
4,7.0,14.8,995.4,9.8,23.0,0.0,4.7,19.4,1.8,15.0,...,9.0,-1.0,6.0,1001.0,0.0,995.4,3.6,15.0,10.4,998.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13874,12.9,19.0,1018.6,6.8,22.0,3.9,6.4,15.8,1.9,12.0,...,7.4,17.0,0.0,1016.4,10.0,1013.9,7.8,5.0,18.0,1013.1
13875,11.8,16.8,1022.0,4.0,3.0,8.7,6.1,15.8,7.5,14.0,...,6.8,5.0,6.0,1016.7,-1.0,1013.5,4.6,0.0,19.0,1013.9
13876,12.1,15.8,1021.5,2.5,10.0,1.9,6.9,16.2,7.4,12.0,...,4.0,0.0,8.0,1018.6,11.0,1018.6,7.4,16.0,16.8,1013.5
13877,11.3,15.8,1021.1,3.4,14.0,7.5,8.3,18.8,8.7,12.0,...,2.5,16.0,6.0,1022.0,0.0,1019.5,6.8,-1.0,15.8,1018.6


## Target: tmax_mañana

In [14]:
metrics = {}
for max_depth in [5, 10, 20, 40, 60, None]:
    start_time = time.time()
    model = DecisionTreeRegressor(max_depth = max_depth)
    model.fit(train[model_columns], train.tmax_mañana);
    
    train_pred = model.predict(train[model_columns])
    test_pred = model.predict(test[model_columns])

    metrics['DT_'+ str(max_depth)] = {
        'Train_MAE': round(mean_absolute_error(train.tmax_mañana, train_pred), 5),
        'Test_MAE': round(mean_absolute_error(test.tmax_mañana, test_pred), 5),
        'Run_Time': time.time() - start_time,
    }

metrics_pd = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_pd['delta'] = metrics_pd.Test_MAE - metrics_pd.Train_MAE
metrics_pd

,Run_Time,Train_MAE,Test_MAE,delta
DT_5,0.484484,1.58803,1.72864,0.14061
DT_10,0.688479,1.23242,1.86727,0.63485
DT_20,0.916564,0.19401,2.30372,2.10971
DT_40,1.003821,0.00004,2.31696,2.31692
DT_60,1.346829,0.00000,2.31349,2.31349
DT_None,1.001367,0.00000,2.31309,2.31309


El delta es la diferencia de MAE's, el Train menos el Test. El resultado es cuántos °C de media se equivoca más en el Test.

In [13]:
metrics_pd.to_parquet('dt_tmax.parquet')

## Target: tmin_mañana

In [15]:
metrics = {}
for max_depth in [5, 10, 20, 40, 60, None]:
    start_time = time.time()
    model = DecisionTreeRegressor(max_depth = max_depth)
    model.fit(train[model_columns], train.tmin_mañana);
    
    train_pred = model.predict(train[model_columns])
    test_pred = model.predict(test[model_columns])

    metrics['DT_'+ str(max_depth)] = {
        'Train_MAE': round(mean_absolute_error(train.tmin_mañana, train_pred), 5),
        'Test_MAE': round(mean_absolute_error(test.tmin_mañana, test_pred), 5),
        'Run_Time': time.time() - start_time,
    }

metrics_pd = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_pd['delta'] = metrics_pd.Test_MAE - metrics_pd.Train_MAE
metrics_pd

,Run_Time,Train_MAE,Test_MAE,delta
DT_5,0.451412,1.44405,1.46611,0.02206
DT_10,0.639395,1.05462,1.47722,0.42260
DT_20,0.917932,0.08743,1.78634,1.69891
DT_40,0.929761,0.00000,1.81665,1.81665
DT_60,0.958153,0.00000,1.82143,1.82143
DT_None,0.978453,0.00000,1.80498,1.80498


El delta es la diferencia de MAE's, el Train menos el Test. El resultado es cuántos °C de media se equivoca más en el Test.

In [16]:
metrics_pd.to_parquet('dt_tmin.parquet')

## Target: prec_mañana

In [17]:
metrics = {}
for max_depth in [5, 10, 20, 40, 60, None]:
    start_time = time.time()
    model = DecisionTreeRegressor(max_depth = max_depth)
    model.fit(train[model_columns], train.prec_mañana);
    
    train_pred = model.predict(train[model_columns])
    test_pred = model.predict(test[model_columns])

    metrics['DT_'+ str(max_depth)] = {
        'Train_MAE': round(mean_absolute_error(train.prec_mañana, train_pred), 5),
        'Test_MAE': round(mean_absolute_error(test.prec_mañana, test_pred), 5),
        'Run_Time': time.time() - start_time,
    }

metrics_pd = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_pd['delta'] = metrics_pd.Test_MAE - metrics_pd.Train_MAE
metrics_pd

,Run_Time,Train_MAE,Test_MAE,delta
DT_5,0.470319,1.34384,1.36495,0.02111
DT_10,0.637297,1.05027,1.68908,0.63881
DT_20,0.942462,0.28541,2.07382,1.78841
DT_40,1.293504,0.00018,2.14677,2.14659
DT_60,1.248676,0.00000,2.02552,2.02552
DT_None,1.315089,0.00000,2.40949,2.40949


El delta es la diferencia de MAE's, el Train menos el Test. El resultado es cuántos mm de media se equivoca más en el Test.

In [18]:
metrics_pd.to_parquet('dt_prec.parquet')

## Target: sol_mañana

In [19]:
metrics = {}
for max_depth in [5, 10, 20, 40, 60, None]:
    start_time = time.time()
    model = DecisionTreeRegressor(max_depth = max_depth)
    model.fit(train[model_columns], train.sol_mañana);
    
    train_pred = model.predict(train[model_columns])
    test_pred = model.predict(test[model_columns])

    metrics['DT_'+ str(max_depth)] = {
        'Train_MAE': round(mean_absolute_error(train.sol_mañana, train_pred), 5),
        'Test_MAE': round(mean_absolute_error(test.sol_mañana, test_pred), 5),
        'Run_Time': time.time() - start_time,
    }

metrics_pd = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_pd['delta'] = metrics_pd.Test_MAE - metrics_pd.Train_MAE
metrics_pd

,Run_Time,Train_MAE,Test_MAE,delta
DT_5,0.473394,2.25559,2.38781,0.13222
DT_10,0.682479,1.81174,2.51466,0.70292
DT_20,1.038242,0.33869,3.00397,2.66528
DT_40,1.071164,0.00000,3.03470,3.03470
DT_60,1.095072,0.00000,3.05946,3.05946
DT_None,1.095590,0.00000,3.14209,3.14209


El delta es la diferencia de MAE's, el Train menos el Test. El resultado es cuántas h de media se equivoca más en el Test.

In [21]:
metrics_pd.to_parquet('dt_sol.parquet')